# Was ist Spark?
https://chartio.com/learn/data-analytics/what-is-spark/

# Ziel: https://spark.apache.org/docs/latest/sql-programming-guide.html
Aufgabe: erstelle ein Programm, das die Zeilen in diesem Sourcecode zaehlt, die das Wort "Spark" enthalten 

In [ ]:
# brauchen wir nicht (zumindest bisher;-)

# erstelle einen Context mit einer bestimmten Configuration
from pyspark import SparkConf, SparkContext
config = SparkConf().setAppName("MySparkApp")
context = SparkContext(conf=config)
# Aufgabe: fuehre diese Zelle 2x aus. Was passiert

In [ ]:
# wir wollen DataFrames nutzen, und dafuer brauchen wir eine SparkSession
# Import SparkSession
from pyspark.sql import SparkSession

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("Datenbanken mit Spark") \
      .getOrCreate() # getOrCreate liefert existierende Session wenn es schon eine gibt

# Aufgabe: fuehre diese Zelle 2x aus. Was passiert? Gibt es einen Unterschied zur Aufgabe davor?
# versuche, die obere Zelle zu "reparieren" damit sie das gleiche Verhalten (das Selbe????) zeigt, wie diese hier

In [ ]:
spark

In [ ]:
# den Inhalt des directories anschauen. Bei mir gibt es ein File "Spark Intro.ipynb")
import os
for subdir, dirs, files in os.walk('./'):
    for file in files:
      print(file)
    
# !ls geht natuerlich schneller;-)

In [ ]:
# dieses File (das wir hier gerade bearbeiten) einlesen
textFile = spark.read.text("Spark Intro.ipynb")

In [ ]:
# wie viele Zeilen ...
textFile.count()

In [ ]:
textFile.first()   # warum kommt da "Mist" raus? eigentlich sollte ja die erste Zeile kommen...

In [ ]:
textFile.show(6)  # nur die ersten 6 Zeilen zeigen

In [ ]:
# jetzt filtern wir die Zeilen mit "Spark" drin raus
linesWithSpark = textFile.filter(textFile.value.contains("Spark"))

# Aufgabe: finde heraus, wie viele Zeilen das Wort "Spark" enthalten,
wie viele den Inhalt "spark" haben, und zeige dann den Inhalt dieses neuen Dataframe an

In [ ]:
textFile.filter(textFile.value.contains("spark")).count()

In [ ]:
# jetzt ein bisschen SQL
from pyspark.sql.functions import *
textFile.select(size(split(textFile.value, "\s+"))).show()

In [ ]:
# wie in SQL koennen wir einer Spalte einen Namen geben: .name("name der Spalte")
# Obacht: das ist Teil des SELECT-statements! (Klammern beachten)
textFile.select(size(split(textFile.value, "\s+")).name("numWords")).show()

In [ ]:
maxWords=textFile.select(size(split(textFile.value, "\s+")).name("numWords")).agg(max(col("numWords")))
textFile.select(size(split(textFile.value, "\s+")).name("numWords")).agg(max(col("numWords"))).collect()

# Aufgabe: extrahiere programmatisch die maximale Anzahl von Worten in einer Zeile

Bsp: https://www.geeksforgeeks.org/pyspark-extracting-single-value-from-dataframe/

was macht .head?

In [ ]:
maxWords.head()[0]

In [ ]:
# was macht "first"?
maxWords.first()['max(numWords)']

man muss wirklich den column-Namen nehmen, in diesem Fall 'max(numWords)'

# Aufgabe: CSV File einlesen

Lies die CSV Datei "studibier.csv" aus Moodle in spark ein, (und berechne das Durchschnittsalter)

In [ ]:
!cat studibier.csv

In [ ]:
df_sbier = spark.read.csv('studibier.csv')

In [ ]:
df_sbier.show()

repariere die komischen header...
https://spark.apache.org/docs/latest/sql-data-sources-csv.html

In [ ]:
df_sbier = spark.read.option('header', 'true').csv('studibier.csv')
df_sbier

In [ ]:
# datentypen anzeigen
if df_sbier.dtypes[1][1] == 'string':
    print("it's a string")

In [ ]:
# describe
df_sbier.describe()
#df_sbier.describe().show()

In [ ]:
df_sbier.show()

In [ ]:
df_sbier.printSchema()

In [ ]:
df_sbier = spark.read.option('header', 'true').csv('studibier.csv', inferSchema=True)
df_sbier

In [ ]:
df_sbier = spark.read.csv('studibier.csv', header=True, inferSchema=True)
df_sbier

# Columns, Rows

In [ ]:
df_sbier.columns

In [ ]:
# zeige nur die verschiedenen Biersorten (A: "distinct" herausfinden)
df_sbier.select(' Lieblingsbier').distinct().show()

In [ ]:
# Aufgabe: finde das Durchschnittsalter ...
# funktioniert avg, oder mean?
# nope...
df_sbier.select(' Alter').mean().show()

# Aufgabe: und wie loest man das jetzt?
mit withColumnRenamed oder Quelldaten fixen

In [ ]:
df_sbier.withColumnRenamed(' Alter', 'Alter').agg({"Alter": "avg"}).show()

In [ ]:
df_sbier.agg({" Alter": "avg"}).show()

In [ ]:
# multiple columns
df_sbier.select([' Lieblingsbier', ' Alter']).show()
# was macht distinct?
# Antwort: es zeigt einzigartige komplette Zeilen (also alle Spalten muessen identisch sein)

In [ ]:
# was passiert denn hier?
from pyspark.sql.functions import lit
df_sbier.withColumn('Alter nach 2 Jahren', df_sbier[' Alter']+2).withColumn('Eins', lit(1)).show()
# https://sparkbyexamples.com/pyspark/pyspark-add-new-column-to-dataframe/

In [ ]:
# ja wo sind sie denn, die schoenen neuen Spalten?
df_sbier.head(5)
# Antwort: da wurde halt in memory geaendert, bzw neu angelegt, das Original wird NICHT geaendert

In [ ]:
df_sbierMitMuell = df_sbier.withColumn('Alter nach 2 Jahren', df_sbier[' Alter']+2).withColumn('Eins', lit(1))
df_sbierMitMuell.show()

In [ ]:
df_sbierMitMuell.drop('Alter nach 2 Jahren').show()

In [ ]:
df_sbierMitMuell.show()

In [ ]:
df_sbierMitMuell.na.drop(how="any",thresh=1).show()

In [ ]:
df_sbierMitMuell.schema

In [ ]:
# Aufgabe: fuege 3 Zeilen hinzu, die 1, 2, oder 3 NULL Werte enthalten
# z.B. https://mungingdata.com/pyspark/none-null/
nullWerteDaten = [('John', 'Weizen', 14.5, None, 2),(None, 'NEIPA', None, None, 2),('Jane','Pils',None,None,2)]
nwdSchema = df_sbierMitMuell.schema
df_bierWithNull = df_sbierMitMuell.union(spark.createDataFrame(data=nullWerteDaten, schema = nwdSchema))

In [ ]:
df_bierWithNull.show()
df_bierWithNull.na.drop(how="any").show()

In [ ]:
#Aufgabe: na.drop entferne Zeilen die 1, 2 oder 3 NULL enthalten
df_bierWithNull.na.drop(how="any",thresh=5).show()

In [ ]:
#Aufgabe: na.drop entferne Zeilen die 1, 2 oder 3 NULL enthalten / ***aber nur wenn das Alter fehlt****
df_bierWithNull.na.drop(how="any",subset=[' Alter']).show()

# Aufgabe: was mache ich denn wenn ich nicht droppen will, sondern ersetzen?
Antwort: mit fill

In [ ]:
# was passiert hier/was ist unerwartet?
df_bierWithNull.na.fill('so nicht').show()
# Antwort: fill schaut sich den Datentyp an und fuellt nur das was passt (int&double kann nicht mit Text gefuellt werden

# Aufgabe 2: fuelle nur "Alter nach 2 Jahren"-Null-Werte ()
df_bierWithNull.na.fill(99,'Alter nach 2 Jahren').show()

# JOINS 
https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/

# Citibike Exercise

In [ ]:
df_citibikeOct = spark.read.option('header','true').csv('/mnt/macbook/spark/202210-citibike-tripdata.csv')

In [ ]:
df_citibikeOct.count()

In [ ]:
type(df_citibikeOct)

In [ ]:
df_citibikeOct.head()